In [1]:
import numpy as np
import pandas as pd

In [31]:
data = pd.read_csv("data_new.csv")

In [3]:
# To find how much missing data is in each column:
# Count the missing values in each column
missing_values = data.isnull().sum()

# Sum the total number of missing values in the entire DataFrame
total_missing_values = missing_values.sum()

# Print the results
print("Number of missing values in each column:")
print(missing_values)
print("\nTotal number of missing values in the DataFrame:", total_missing_values)

Number of missing values in each column:
id                     0
risk_score_t           0
program_enrolled_t     0
cost_t                 0
cost_avoidable_t       0
                      ..
trig_max-low_tm1       0
trig_max-high_tm1      0
trig_max-normal_tm1    0
gagne_sum_tm1          0
gagne_sum_t            0
Length: 161, dtype: int64

Total number of missing values in the DataFrame: 127857


In [ ]:
data_drop = data.dropna()

If we drop all the NaN values, there will be an imbalanced number of white and black subjects, therefore, we will perform imputation.

In [11]:
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.pipeline import Pipeline



In [32]:
race_mapping = {'black': 0, 'white': 1}

# Apply the mapping to the 'Race' column
data['race'] = data['race'].map(race_mapping)
data_use = data.drop(columns = ["id","program_enrolled_t", "cost_t", "cost_avoidable_t", "cost_emergency_tm1", "cost_home_health_tm1", "cost_ip_medical_tm1", "cost_ip_surgical_tm1", "cost_laboratory_tm1", "cost_op_primary_care_tm1", "cost_op_specialists_tm1", "cost_op_surgery_tm1", "cost_other_tm1", "cost_pharmacy_tm1", "cost_physical_therapy_tm1", "cost_radiology_tm1"])

In [36]:
def evaluate_imputation(X, y, imputer, model):
    print(np.sum(np.isnan(X), axis = 0).sum())
    X_imputed = imputer.fit_transform(X)
    # To check that imputation worked
    missing_values = np.sum(np.isnan(X_imputed), axis=0)

    print("Number of missing values: ", missing_values.sum())
    X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    return mse

In [37]:
mean_imputer = SimpleImputer(strategy='mean')
mean_mse = evaluate_imputation(data_use.drop('risk_score_t', axis=1), data_use['risk_score_t'], mean_imputer, LinearRegression())
print(f"Mean Imputation MSE: {mean_mse}")

127857
Number of missing values:  0
Mean Imputation MSE: 14.202401501649728


In [38]:
# Median imputation
median_imputer = SimpleImputer(strategy='median')
median_mse = evaluate_imputation(data_use.drop('risk_score_t', axis=1), data_use['risk_score_t'], median_imputer, LinearRegression())
print(f"Median Imputation MSE: {median_mse}")

127857
Number of missing values:  0
Median Imputation MSE: 14.189681550319609


In [26]:
knn_imputer = KNNImputer(n_neighbors=5)
knn_mse = evaluate_imputation(data_use.drop('risk_score_t', axis=1), data_use['risk_score_t'], knn_imputer, LinearRegression())
print(f"KNN Imputation MSE: {knn_mse}")

176641
Number of missing values:  0
KNN Imputation MSE: 13.827733774434314


The KNN imputation strategy seems to have the lowest loss and provide the closest estimate. 


In [39]:
X_imputed = knn_imputer.fit_transform(data_use)
imputed_df = pd.DataFrame(X_imputed, columns = data_use.columns)

In order to ensure computational efficiency, we are only going to use a subset of 10,000 samples from this imputed data, and it will be race balanced to see if race is a predictor of outcome

In [40]:
# Calculate the proportion of each race in the sample
desired_samples_per_race = 5000

# Initialize an empty DataFrame to store the balanced sample
balanced_sample_df = pd.DataFrame()

# Sample a fixed number of entries for each race to ensure balance
for race in imputed_df['race'].unique():
    race_entries = imputed_df[imputed_df['race'] == race].sample(n=desired_samples_per_race, random_state=42)
    balanced_sample_df = pd.concat([balanced_sample_df, race_entries])

# Shuffle the balanced sample to randomize the order
balanced_sample_df = balanced_sample_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the race distribution in the balanced sample
balanced_race_distribution = balanced_sample_df['race'].value_counts()
print("\nBalanced Race Distribution in the Sample:")
print(balanced_race_distribution)


Balanced Race Distribution in the Sample:
0.0    5000
1.0    5000
Name: race, dtype: int64


In [41]:
balanced_sample_df.to_csv("balanced_df.csv")

In [ ]:
values = [5.13652409e-02,7.98255124e-02,6.33596779e-02,6.14203773e-02,6.62250823e-02,1.33299583e-02,1.03585403e-02,3.05865491e-03,7.92639697e-03,9.32028083e-03,1.03305909e-02,9.11003831e-03,1.70765062e-02,1.85074418e-02,8.50836660e-04,4.71321642e-03,6.62586789e-03,4.16496345e-03,1.27823454e-04,1.04145764e-03,1.97118637e-03,4.80048442e-03,8.10138517e-04,3.02331188e-03,3.39673144e-02,4.29393672e-03,1.73655544e-03,3.07980103e-03,6.93691128e-03,1.08537190e-04,3.02503575e-03,5.21722310e-04,1.74392798e-03,3.29903769e-03,1.17473252e-02,2.83653085e-03,1.76932158e-04,1.54087510e-03,1.71207632e-03,9.67688299e-04,4.33602906e-04,5.44887010e-04,1.36422030e-03,1.25001249e-03,7.85808475e-03,1.44214201e-02,3.97312557e-04,4.17278565e-03,1.07840771e-03,1.05081474e-03,1.30353110e-03,1.15013211e-03,3.42663495e-02,8.27169452e-06,4.00974613e-03,1.09504399e-02,2.38082538e-02,8.22585510e-03,1.76240769e-03,3.24994512e-02,7.29197012e-03,3.96417740e-03,2.15381392e-03,3.95493097e-03,3.66517185e-03,2.72390801e-03,3.91747151e-03,3.63032273e-03,4.94403167e-03,4.19256340e-03,0.00000000e+00,0.00000000e+00,0.00000000e+00,0.00000000e+00,0.00000000e+00,0.00000000e+00,0.00000000e+00,0.00000000e+00,0.00000000e+00,0.00000000e+00,1.35451846e-03,2.09112530e-03,0.00000000e+00,1.70440219e-03,2.10849659e-03,0.00000000e+00,1.51163817e-03,2.13259127e-03,2.37852222e-05,3.78471613e-03,3.97334073e-03,3.25309131e-05,5.97024155e-03,3.74337490e-03,1.90025493e-05,7.81035511e-03,3.69272215e-03,3.51140276e-03,4.52120255e-04,5.57463904e-03,2.18001687e-03,5.58353139e-04,5.76305517e-03,1.65728488e-03,5.71980590e-04,5.91363081e-03,9.40973758e-04,4.68451392e-03,5.41890141e-03,6.77386608e-04,4.34573509e-03,3.67491644e-03,5.62753691e-04,4.78276804e-03,3.39377808e-03,4.05217383e-04,4.91462168e-04,3.90193437e-04,4.83171868e-04,4.96152381e-04,3.52796097e-04,4.99416278e-04,4.32399595e-04,3.35588464e-04,2.41999557e-03,7.19437898e-05,5.74586962e-03,1.05422373e-03,6.69420390e-05,6.14621151e-03,6.96958002e-04,5.46125696e-04,9.17919475e-03,2.54014586e-03,2.84579124e-03,4.26919911e-03,1.94743409e-03,3.00953496e-03,4.41133456e-03,1.95206652e-03,3.12370193e-03,4.26644423e-03,7.07181382e-02,5.84531535e-02]

In [ ]:
from matplotlib import pyplot as pl
data = list(zip(labels, values))
data.sort(key=lambda x: x[1], reverse=True)

# Select the top 10 values
top_labels, top_values = zip(*data[:15])

# Create a bar graph for the top 10 values
plt.figure(figsize=(15, 8))  # Adjust width and height as needed

plt.bar(top_labels, top_values)
plt.xticks(rotation=90)  # Adjust the rotation angle as needed
plt.title("Feature Importances for Best Model", size = 14)
plt.xlabel("Features", size = 14)
plt.ylabel("Feature Importance Value", size = 14)
plt.rc('font', size=12)